In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import glob
import time

In [2]:
#连接年月日，统一格式%Y-%m-%d
def concat_date(dvsdf,concatname):
    YMD=[]
    for i in range(dvsdf.shape[0]):
        ymd=str(dvsdf['年份'][i])+'-'+str(dvsdf[concatname][i]).zfill(4)[:2]+'-'+str(dvsdf[concatname][i])[-2:]
        YMD.append(ymd)
    return YMD

#计算两个日期之间的间隔天数
import datetime
def date_period(time1,time2):
    a1 = time1
    a2 = time2
    b1 = a1.split('-')
    b2 = a2.split('-')
    c1=datetime.date(int(b1[0]), int(b1[1]), int(b1[2]))   #开始的日期de
    c2=datetime.date(int(b2[0]), int(b2[1]), int(b2[2]))  #  结束的日期
    result=(c2-c1).days
    return result

In [3]:
#计算日序DOY
import datetime
def DOY_cal(dvsdf,time):
    DOY=[]
    for i in range(dvsdf.shape[0]):
        a=datetime.datetime.strptime(time[i],'%Y-%m-%d').strftime('%Y%m%d')
        year=int(a[:4])
        month=int(a[4:6])
        day=int(a[6:])

        day_of_second = 29 if year % 4 == 0 and year % 100 != 0 or year % 400 == 0 else 28
        days_of_month = (31, day_of_second, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31)

        total_days = sum(days_of_month[: month - 1])
        total_days += day
        DOY.append(total_days)
    return DOY

#计算日长
import math
def daylength(lat,DOY):
    rad=0.017453292
    dec=-math.asin(math.sin(23.45*rad)*math.cos(2*math.pi*(DOY+10)/365))
    sinld=math.sin(rad*lat)*math.sin(dec)
    cosld=math.cos(rad*lat)*math.cos(dec)
    aob=sinld/cosld
    if (aob < -1 ):
        D=0
    elif (aob > 1):
        D=24
    else:
        D=12*(1+2*math.asin(aob)/math.pi) 
    D=D+0.9    #d应为天文日长，考虑到暮曙光作用，因此加上日出前后各6度，折算为小时共0.8h
    return D

In [4]:
#计算逐日热时DTT
import math
def calculate_DTT(mete):
    DTT=[]
    for i in range(mete.shape[0]):
        HU=0.00
        for h in range(1,25,1):#一天24个小时，取不到24，右边加1到25
            Td=(float(mete['TemMax'][i])+float(mete['TemMin'][i]))/2+(float(mete['TemMax'][i])-float(mete['TemMin'][i]))*math.cos(0.2618*(h-14))/2
            if Td > 8 and Td < 30:
                HUH=(Td-8)/24
            elif Td > 30 and Td < 42:
                HUH=(30-(Td-30)*(30-8)/(42-30))/24
            else:
                HUH=0.00
            HU=HU+HUH
        DTT.append(HU)
    return DTT  

#计算逐日热时DTT
# import math
# def calculate_DTT(mete):
#     HU=0.00
#     for h in range(1,25,1):#一天24个小时，取不到24，右边加1到25
#         Td=(float(mete['TemMax'])+float(mete['TemMin']))/2+(float(mete['TemMax'])-float(mete['TemMin']))*math.cos(0.2618*(h-14))/2
#         if Td > 8 and Td < 30:
#             HUH=(Td-8)/24
#         elif Td > 30 and Td < 42:
#             HUH=(30-(Td-30)*(30-8)/(42-30))/24
#         else:
#             HUH=0.0
#         HU=HU+HUH
#     return float(HU)        

In [6]:
dvsfilepath=sorted(glob.glob("D:/Wu/小论文/汕优63物候/1.汕优63物候DVS/DVSDF/*csv"))
metefilepath=sorted(glob.glob("D:/Wu/小论文/Meteo/*txt"))
df=pd.read_excel("D:/Wu/小论文/汕优63物候/汕优63站点目录.xls")

In [7]:
#参数取值范围参考源代码中的
DVRJ_L   = 0.0000  #0.0001
DVRJ_H   = 0.0026  #0.0025
DVRI_L   = 0.0000  #0.0001
DVRI_H   = 0.0026  #0.0025
DVRP_L   = 0.0000  #0.0001
DVRP_H   = 0.0026  #0.0025
DVRR_L   = 0.0000  #0.0001
DVRR_H   = 0.005   #0.005
SHCKD_L  = 0.0     #0.1
SHCKD_H  = 0.7     #0.7
PPSE_L   = 0.0     #0.0
PPSE_H   = 0.6     #0.5
MOPP     = 11.5    #11.5

#温度效应的三基点温度。
Tb   = 8.
Topt = 30.
Tmax = 42.

#共1个站：54362(41.67)
#StNum = '54362'
#Lat   = 41.67

按ORYZA2000模型，只取4个发育期，分别是1出苗、2移栽、3抽穗和4成熟
1.出苗-抽穗寻参

In [8]:
for d,m in zip(dvsfilepath,metefilepath):#表示对‘站号’文件进行遍历
    print("正在进行"+d[-13:-4]) 
    ID=pd.to_numeric(d[-13:-8])
    dvsdf=pd.read_csv(d,encoding='gbk')
    mete=pd.read_table(m,encoding='gbk',sep=' * ',engine='python')
    mete=mete.drop(index=0,axis=1) #去掉第一行
    mete['ymd']=mete['YY']+'-'+mete['mm']+'-'+mete['dd']
    mete.index=pd.DatetimeIndex(mete['ymd'])
    mete['DTT']=calculate_DTT(mete)
    f2=defaultdict(list)
    for DVRJ_1 in np.arange(DVRJ_L,DVRJ_H, 0.0005):  #开始DVRJ循环，出苗-基本营养生长期

        for DVRI_1 in np.arange(DVRI_L,DVRI_H, 0.0005):  #开始DVRI循环，基本营养生长期-幼穗分化始期

            for PPSE_1 in np.arange(PPSE_L,PPSE_H, 0.1):  #开始PPSE循环，光敏感系数

                for DVRP_1 in np.arange(DVRP_L,DVRP_H, 0.0005):  #开始DVRP循环，幼穗分化始期-开花期

                    for SHCKD_1 in np.arange(SHCKD_L,SHCKD_H, 0.1):  #开始SHCKD循环
                        f1=defaultdict(list)

                        #From emergence to heading
                        for i in range(dvsdf.shape[0]):  #开始逐年循环 
                            DVSRate = 0.
                            CROPSTA = 0
                            TSTR    = 0.
                            TS = 0.
                            TSHCKD = 0.

                            n=(pd.Timestamp(dvsdf['emergence'][i])+pd.Timedelta("1day")).date().strftime('%Y-%m-%d')  #原始数据   字符串切片只是为了去掉时间格式中的时分秒
                            m=(pd.Timestamp(dvsdf['heading'][i])+pd.Timedelta("30day")).date().strftime('%Y-%m-%d')  #误差不超过30天 
                            period1=date_period(n,m)   #模拟误差最大不能超过30天
                            period_d1=0 
                            day1=pd.Timestamp(dvsdf['emergence'][i])
                            Tspdate=(pd.Timestamp(dvsdf["transplanting"][i])).date().strftime('%Y-%m-%d')
                            DOY=dvsdf['DOY2'][i]
                            lat=df[df['站号']==ID]["纬度"]

                            for kk in range(period1+1):   #开始该生长季内的逐日循环
                                daystr1=(day1+pd.Timedelta(kk,unit='d')).date().strftime('%Y-%m-%d')
                                DL=daylength(lat,DOY)
                                metedf=mete[mete.index==daystr1]
                                HU = float(metedf["DTT"])
                                if daystr1 == Tspdate:
                                    CROPSTA = 3
                                elif daystr1 > Tspdate:
                                    CROPSTA = 4

                                if DVSRate >= 0. and DVSRate < 0.40:
                                    DVR = DVRJ_1 * HU

                                if DVSRate >= 0.40 and DVSRate < 0.65:
                                    PPFAC = 0.
                                    if DL < MOPP:
                                        PPFAC = 1.
                                    else :
                                        PPFAC = 1. - (DL - MOPP) * PPSE_1
                                    PPFAC = np.min([1.,np.max([0., PPFAC])])
                                    DVR   = DVRI_1 * HU * PPFAC

                                if DVSRate >= 0.65 and DVSRate < 1.00 :
                                    DVR = DVRP_1 * HU

                                if(CROPSTA == 3):
                                    TSTR = TS
                                    TSHCKD = SHCKD_1*TSTR+TSTR

                                if(CROPSTA > 3 and TS < (TSHCKD) ):
                                    DVR = 0.
                                #print("日期：",daystr1,
        #                               "CROPSTA = ",CROPSTA,"TS = ", TS, "TSHCKD = ",TSHCKD
        #                               "DVR=",DVR,"DVSRate = ",DVSRate)
                                TS = TS + HU 
                                DVSRate = DVSRate + DVR
                                period_d1+=1
                                DOY+=1

                                if (DVSRate >= 1.0 ):
                                    #print("抽穗日：",daystr1,period_d1,"到达抽穗日的DVS：",DVSRate)
                                    Error1 = period_d1-dvsdf["phase_EH"][i]
                                    f1["误差"].append(Error1)
                                    #print(Error)
                                    break
                            if (DVSRate < 1.0 ):
                                f1["误差"].append(np.nan)
                            #结束逐年循环\n",
                        F1=pd.DataFrame(f1)
                        F1=F1.dropna(axis=0)

                        if F1.shape[0]!=0: 
                            s=0
                            ny=len(F1)
                            for y in F1["误差"]:
                                s+=y**2
                            RMSE=(s/F1.shape[0])**0.5  
                            TSumJ_1 = 1/DVRJ_1      #TSum in BVP phase
                            TSumI_1 = 1/DVRI_1      #TSum in PVP phase
                            TSumP_1 = 1/DVRP_1      #TSum in PFP phase
                            f2["DVRJ"].append(DVRJ_1)
                            f2["DVRI"].append(DVRI_1)
                            f2["PPSE"].append(PPSE_1)
                            f2["DVRP"].append(DVRP_1)
                            f2["SHCKD"].append(SHCKD_1)
                            f2["TSum_BVP"].append(TSumJ_1)
                            f2["TSum_PVP"].append(TSumI_1)
                            f2["TSum_PFP"].append(TSumP_1)
                            f2["RMSE"].append(RMSE)
                            f2["预测年数"].append(ny)
                            #print(DVRJ_1,DVRI_1,PPSE_1,DVRP_1,RMSE,ny)        
    F2=pd.DataFrame(f2)
    name1="D:/Wu/小论文/oryza2000站点参数/"+d[-13:-4]+'参数表.csv'
    F2.to_csv(name1,index=False,encoding='gbk')
    print(d[-13:-4],"执行结束  √","     完成时间",datetime.datetime.now())   
print("程序运行结束！")

正在进行56280四川名山
56280四川名山 执行结束  √      完成时间 2021-12-31 12:06:40.459481
正在进行56491四川宜宾
56491四川宜宾 执行结束  √      完成时间 2021-12-31 13:03:33.092015
正在进行56844云南芒市
56844云南芒市 执行结束  √      完成时间 2021-12-31 14:03:11.745261
正在进行56946云南耿马
56946云南耿马 执行结束  √      完成时间 2021-12-31 14:56:50.338156
正在进行56959云南景洪
56959云南景洪 执行结束  √      完成时间 2021-12-31 15:45:38.542299
正在进行57206四川广元
57206四川广元 执行结束  √      完成时间 2021-12-31 16:44:43.969280
正在进行57249湖北竹溪
57249湖北竹溪 执行结束  √      完成时间 2021-12-31 18:40:09.468556
正在进行57251湖北郧西
57251湖北郧西 执行结束  √      完成时间 2021-12-31 20:40:02.587259
正在进行57259湖北房县
57259湖北房县 执行结束  √      完成时间 2021-12-31 23:08:18.839893
正在进行57268湖北谷城
57268湖北谷城 执行结束  √      完成时间 2022-01-01 01:27:59.620297
正在进行57313四川巴中
57313四川巴中 执行结束  √      完成时间 2022-01-01 02:25:19.278563
正在进行57378湖北钟祥
57378湖北钟祥 执行结束  √      完成时间 2022-01-01 04:51:53.473676
正在进行57381湖北随州
57381湖北随州 执行结束  √      完成时间 2022-01-01 06:40:46.704965
正在进行57420四川大竹
57420四川大竹 执行结束  √      完成时间 2022-01-01 07:42:39.818079
正在进行57482湖北孝感
57482湖北孝感 执行结束  √   

In [13]:
parafile=sorted((glob.glob("D:/Wu/小论文/oryza2000站点参数/emergence-heading/*csv")))
f4=defaultdict(list)

for p in parafile:
    paradf=pd.read_csv(p,encoding="gbk",sep=",")
    ID=paradf[-13:-4]
    co=paradf[paradf["预测年数"]==paradf["预测年数"].max()]
    a=co[co["RMSE"]==co["RMSE"].min()]
    if len(a)>1 :
        b=a[a["DVRJ"]<=a["DVRP"]]
        if len(b)>1 :
            c=b[b["DVRI"]<=b["DVRP"]]
            if  len(c)>1 :
                d=c[c["PPSE"]==c["PPSE"].min()]
                if len(d)>1 :
                    f=d[d["SHCKD"]==d["SHCKD"].min()]
                    f4["站号"].append(int(p[-16:-11]))
                    f4["最优-DVRJ"].append(float(f["DVRJ"]))
                    f4["最优-DVRI"].append(float(f["DVRI"]))
                    f4["最优-DVRP"].append(float(f["DVRP"]))
                    f4["最优-PPSE"].append(float(f["PPSE"]))
                    f4["SHCKD"].append(float(f["SHCKD"]))
                    f4["TSum_BVP"].append(float(f["TSum_BVP"]))
                    f4["TSum_PVP"].append(float(f["TSum_PVP"]))
                    f4["TSum_PFP"].append(float(f["TSum_PFP"]))
                    f4["RMSE"].append(float(f["RMSE"]))
                else:
                    f4["站号"].append(int(p[-16:-11]))
                    f4["最优-DVRJ"].append(float(d["DVRJ"]))
                    f4["最优-DVRI"].append(float(d["DVRI"]))
                    f4["最优-DVRP"].append(float(d["DVRP"]))
                    f4["最优-PPSE"].append(float(d["PPSE"]))
                    f4["SHCKD"].append(float(d["SHCKD"]))
                    f4["TSum_BVP"].append(float(d["TSum_BVP"]))
                    f4["TSum_PVP"].append(float(d["TSum_PVP"]))
                    f4["TSum_PFP"].append(float(d["TSum_PFP"]))
                    f4["RMSE"].append(float(d["RMSE"]))
            else:
                f4["站号"].append(int(p[-16:-11]))
                f4["最优-DVRJ"].append(float(c["DVRJ"]))
                f4["最优-DVRI"].append(float(c["DVRI"]))
                f4["最优-DVRP"].append(float(c["DVRP"]))
                f4["最优-PPSE"].append(float(c["PPSE"]))
                f4["SHCKD"].append(float(c["SHCKD"]))
                f4["TSum_BVP"].append(float(c["TSum_BVP"]))
                f4["TSum_PVP"].append(float(c["TSum_PVP"]))
                f4["TSum_PFP"].append(float(c["TSum_PFP"]))
                f4["RMSE"].append(float(c["RMSE"]))
        else:
            f4["站号"].append(int(p[-16:-11]))
            f4["最优-DVRJ"].append(float(b["DVRJ"]))
            f4["最优-DVRI"].append(float(b["DVRI"]))
            f4["最优-DVRP"].append(float(b["DVRP"]))
            f4["最优-PPSE"].append(float(b["PPSE"]))
            f4["SHCKD"].append(float(b["SHCKD"]))
            f4["TSum_BVP"].append(float(b["TSum_BVP"]))
            f4["TSum_PVP"].append(float(b["TSum_PVP"]))
            f4["TSum_PFP"].append(float(b["TSum_PFP"]))
            f4["RMSE"].append(float(b["RMSE"]))
    else:
        f4["站号"].append(int(p[-16:-11]))
        f4["最优-DVRJ"].append(float(a["DVRJ"]))
        f4["最优-DVRI"].append(float(a["DVRI"]))
        f4["最优-DVRP"].append(float(a["DVRP"]))
        f4["最优-PPSE"].append(float(a["PPSE"]))
        f4["SHCKD"].append(float(a["SHCKD"]))
        f4["TSum_BVP"].append(float(a["TSum_BVP"]))
        f4["TSum_PVP"].append(float(a["TSum_PVP"]))
        f4["TSum_PFP"].append(float(a["TSum_PFP"]))
        f4["RMSE"].append(float(a["RMSE"]))

    print(p[-16:-7],"执行结束  √","     完成时间",datetime.datetime.now())
F4=pd.DataFrame(f4)
print(F4)
name ="D:/Wu/小论文/oryza2000站点参数/寻参结果"+'VGP阶段参数.csv'
F4.to_csv(name,index=False,encoding='gbk')


56280四川名山 执行结束  √      完成时间 2022-01-02 21:57:21.028543
56491四川宜宾 执行结束  √      完成时间 2022-01-02 21:57:21.037518
56844云南芒市 执行结束  √      完成时间 2022-01-02 21:57:21.046494
56946云南耿马 执行结束  √      完成时间 2022-01-02 21:57:21.054472
56959云南景洪 执行结束  √      完成时间 2022-01-02 21:57:21.063448
57206四川广元 执行结束  √      完成时间 2022-01-02 21:57:21.071427
57249湖北竹溪 执行结束  √      完成时间 2022-01-02 21:57:21.079406
57251湖北郧西 执行结束  √      完成时间 2022-01-02 21:57:21.088381
57259湖北房县 执行结束  √      完成时间 2022-01-02 21:57:21.095363
57268湖北谷城 执行结束  √      完成时间 2022-01-02 21:57:21.106334
57313四川巴中 执行结束  √      完成时间 2022-01-02 21:57:21.117304
57378湖北钟祥 执行结束  √      完成时间 2022-01-02 21:57:21.128275
57381湖北随州 执行结束  √      完成时间 2022-01-02 21:57:21.138248
57420四川大竹 执行结束  √      完成时间 2022-01-02 21:57:21.147225
57482湖北孝感 执行结束  √      完成时间 2022-01-02 21:57:21.155204
57517四川江津 执行结束  √      完成时间 2022-01-02 21:57:21.163181
57523四川丰都 执行结束  √      完成时间 2022-01-02 21:57:21.172158
57581湖北洪湖 执行结束  √      完成时间 2022-01-02 21:57:21.181134
57717贵州遵义 

In [14]:
parafile=sorted((glob.glob("D:/Wu/小论文/oryza2000站点参数/heading-maturity parameters/*csv")))
f4=defaultdict(list)
for p in parafile:
    paradf=pd.read_csv(p,encoding="gbk",sep=",")
    co=paradf[paradf["预测年数"]==paradf["预测年数"].max()]
    a=co[co["RMSE"]==co["RMSE"].min()]
    f4["站号"].append(int(p[-16:-11]))
    f4["最优-DVRR"].append(float(a["DVRR"]))
    f4["最优-积温"].append(int(a["TSum_GFP"]))
    f4["RMSE"].append(float(a["RMSE"]))

    print(p[-16:-7],"执行结束  √","     完成时间",datetime.datetime.now())
F4=pd.DataFrame(f4)
print(F4)
name ="D:/Wu/小论文/oryza2000站点参数/寻参结果"+'RGP阶段参数.csv'
F4.to_csv(name,index=False,encoding='gbk')

56280四川名山 执行结束  √      完成时间 2022-01-02 21:59:06.929728
56491四川宜宾 执行结束  √      完成时间 2022-01-02 21:59:06.932577
56844云南芒市 执行结束  √      完成时间 2022-01-02 21:59:06.934395
56946云南耿马 执行结束  √      完成时间 2022-01-02 21:59:06.936850
56959云南景洪 执行结束  √      完成时间 2022-01-02 21:59:06.939843
57206四川广元 执行结束  √      完成时间 2022-01-02 21:59:06.941837
57249湖北竹溪 执行结束  √      完成时间 2022-01-02 21:59:06.943832
57251湖北郧西 执行结束  √      完成时间 2022-01-02 21:59:06.946824
57259湖北房县 执行结束  √      完成时间 2022-01-02 21:59:06.948818
57268湖北谷城 执行结束  √      完成时间 2022-01-02 21:59:06.951810
57313四川巴中 执行结束  √      完成时间 2022-01-02 21:59:06.954803
57378湖北钟祥 执行结束  √      完成时间 2022-01-02 21:59:06.956797
57381湖北随州 执行结束  √      完成时间 2022-01-02 21:59:06.958792
57420四川大竹 执行结束  √      完成时间 2022-01-02 21:59:06.960787
57482湖北孝感 执行结束  √      完成时间 2022-01-02 21:59:06.962782
57517四川江津 执行结束  √      完成时间 2022-01-02 21:59:06.965773
57523四川丰都 执行结束  √      完成时间 2022-01-02 21:59:06.967767
57581湖北洪湖 执行结束  √      完成时间 2022-01-02 21:59:06.969762
57717贵州遵义 

In [10]:
paradf=pd.read_csv('D:/Wu/小论文/oryza2000站点参数/emergence-heading/57523四川丰都参数表.csv',encoding="gbk",sep=",")
co=paradf[paradf["预测年数"]==paradf["预测年数"].max()]
a=co[co["RMSE"]==co["RMSE"].min()]
if len(a)>1 :
        b=a[a["DVRJ"]>=a["DVRP"]]
len(b)
a

,DVRJ,DVRI,PPSE,DVRP,SHCKD,TSum_BVP,TSum_PVP,TSum_PFP,RMSE,预测年数
244,0.0005,0.0015,0.2,0.0010,0.1,2000.0,666.666667,1000.000000,1.414214,5
253,0.0005,0.0015,0.2,0.0015,0.3,2000.0,666.666667,666.666667,1.414214,5
360,0.0005,0.0020,0.2,0.0015,0.5,2000.0,500.000000,666.666667,1.414214,5


In [ ]:
co=F2[F2["预测年数"]==F2["预测年数"].max()]
a=co[co["RMSE"]==co["RMSE"].min()]
a

In [ ]:
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

2.抽穗-成熟期寻参，期间发育速率为DVRR_1

In [ ]:
f4=defaultdict(list)
for DVRR_1 in np.arange(DVRR_L, DVRR_H, 0.0001)  #开始DVRR循环，开花-成熟
    f3=defaultdict(list)
    for i in range(df.shape[0]):  #开始逐年循环
        n=(pd.Timestamp(df['heading'][i])+pd.Timedelta("1day")).date().strftime('%Y-%m-%d')  #原始数据   字符串切片只是为了去掉时间格式中的时分秒
        m=(pd.Timestamp(df['maturity'][i])+pd.Timedelta("30day")).date().strftime('%Y-%m-%d')  #误差不超过30天 
        period1=date_period(n,m)   #模拟误差最大不能超过30天
        period_d1=0 
        day1=pd.Timestamp(df['heading'][i])
        DOY=df['DOY4'][i]
        DVSRate = 0
        for k in range(period1+1):   #开始该生长季内的逐日循环
            daystr1=(day1+pd.Timedelta(kk,unit='d')).date().strftime('%Y-%m-%d')
            DL=daylength(41.6667,DOY)
            metedf=mete[mete.index==daystr1]
            HU = float(metedf["DTT"])
            DVR = DVRR_1 * HU
            if (DVSRate .ge. 1.) then
                MaturityError = period_d1-df["phase_HM"][i]
                f3["误差"].append(MaturityError)
                break
            DVSRate = DVSRate + max(DVR, 0.)
            period_d1+=1
            DOY+=1
            
        if (DVSRate < 1.0 ):
            f3["误差"].append(np.nan)
            #结束逐年循环\n",
    F3=pd.DataFrame(f3)
    F3=F3.dropna(axis=0)

    if F3.shape[0]==0: 
        break
    else:
        s=0
        ny=len(F3)
        for y in F3["误差"]:
            s+=y**2
        RMSE=(s/F3.shape[0])**0.5  
        TSumR_1 = 1/DVRR_1      #TSum in GFP phase
        f4["DVRR"].append(DVRR_1)
        f4["TSum_GFP"].append(TSumR_1)
        f4["RMSE"].append(RMSE)
        f4["预测年数"].append(ny)
        #print(DVRJ_1,DVRI_1,PPSE_1,DVRP_1,RMSE,ny)        
F4=pd.DataFrame(f4)   

----------------  模拟出苗-抽穗   ---------------

In [45]:
DVRJ=0.00185  #出苗-基本营养生长期
DVRI=0.00235  #基本营养生长期-幼穗分化始期
PPSE=0.2  #光敏感系数
DVRP=0.0021  #幼穗分化始期-开花期
SHCKD=0.5  #移栽效应
MOPP=11.5  #最佳光周期
f4=defaultdict(list)
                    
#From emergence to heading
for i in range(df.shape[0]):  #开始逐年循环 
    DVSRate = 0.
    CROPSTA = 0
    TSTR    = 0.
    TS = 0.
    TSHCKD = 0.

    n=(pd.Timestamp(df['emergence'][i])+pd.Timedelta("1day")).date().strftime('%Y-%m-%d')  #原始数据   字符串切片只是为了去掉时间格式中的时分秒
    m=(pd.Timestamp(df['heading'][i])+pd.Timedelta("30day")).date().strftime('%Y-%m-%d')  #误差不超过30天 
    period1=date_period(n,m)   #模拟误差最大不能超过30天
    period_d1=0 
    day1=pd.Timestamp(df['emergence'][i])
    Tspdate=(pd.Timestamp(df["transplanting"][i])).date().strftime('%Y-%m-%d')
    DOY=df['DOY1'][i]

    for kk in range(period1+1):   #开始该生长季内的逐日循环
        daystr1=(day1+pd.Timedelta(kk,unit='d')).date().strftime('%Y-%m-%d')
        DL=daylength(41.6667,DOY)
        metedf=mete[mete.index==daystr1]
        HU = float(metedf["DTT"])
        if daystr1 == Tspdate:
            CROPSTA = 3
        elif daystr1 > Tspdate:
            CROPSTA = 4

        if DVSRate >= 0. and DVSRate < 0.40:
            DVR = DVRJ * HU

        if DVSRate >= 0.40 and DVSRate < 0.65:
            PPFAC = 0.
            if DL < MOPP:
                PPFAC = 1.
            else :
                PPFAC = 1. - (DL - MOPP) * PPSE
            PPFAC = np.min([1.,np.max([0., PPFAC])])
            DVR   = DVRI * HU * PPFAC

        if DVSRate >= 0.65 and DVSRate < 1.00 :
            DVR = DVRP * HU

        if(CROPSTA == 3):
            TSTR = TS
            TSHCKD = SHCKD*TSTR+TSTR

        if(CROPSTA > 3 and TS < (TSHCKD) ):
            DVR = 0.
        #print("日期：",daystr1,
#              "CROPSTA = ",CROPSTA,"TS = ", TS, "TSHCKD = ",TSHCKD
#               "DVR=",DVR,"DVSRate = ",DVSRate)
        TS = TS + HU 
        DVSRate = DVSRate + DVR
        period_d1+=1
        DOY+=1

        if (DVSRate >= 1.0 ):
            #print("抽穗日：",daystr1,period_d1,"到达抽穗日的DVS：",DVSRate)
            Errorheading = period_d1-df["phase_EH"][i]
            f4["误差"].append(Errorheading)
            #print(Error)
            break
    if (DVSRate < 1.0 ):
        f4["误差"].append(np.nan)
        #生长季内的逐日循环结束
    f4["年份"].append(df["年份"][i])
F4=pd.DataFrame(f4)
F4.to_csv('D:/Wu/54362/Errorheading.csv',index=False,encoding='gbk')
if F4.shape[0]!=0: 
    s=0
    ny=len(F4)
    for y in F4["误差"]:
        s+=y**2
    RMSE=(s/F4.shape[0])**0.5  
#     TSumJ = 1/DVRJ      #TSum in BVP phase
#     TSumI = 1/DVRI      #TSum in PVP phase
#     TSumP = 1/DVRP      #TSum in PFP phase
#     f2["TSum_BVP"].append(TSumJ_1)
#     f2["TSum_PVP"].append(TSumI_1)
#     f2["TSum_PFP"].append(TSumP_1)
    print(RMSE,ny)        


2.7386127875258306 26


----------------  模拟出苗-成熟   ---------------

In [53]:
DVRJ=0.00185  #出苗-基本营养生长期
DVRI=0.00235  #基本营养生长期-幼穗分化始期
PPSE=0.2  #光敏感系数
DVRP=0.0021  #幼穗分化始期-开花期
SHCKD=0.5  #移栽效应
DVRR=0.002068  #开花-成熟
MOPP=11.5  #最佳光周期
f1=defaultdict(list)
                    
#From emergence to heading
for i in range(df.shape[0]):  #开始逐年循环 
    DVSRate = 0.
    CROPSTA = 0
    TSTR    = 0.
    TS = 0.
    TSHCKD = 0.

    n=(pd.Timestamp(df['emergence'][i])+pd.Timedelta("1day")).date().strftime('%Y-%m-%d')  #原始数据   字符串切片只是为了去掉时间格式中的时分秒
    m=(pd.Timestamp(df['maturity'][i])+pd.Timedelta("30day")).date().strftime('%Y-%m-%d')  #误差不超过30天 
    period1=date_period(n,m)   #模拟误差最大不能超过30天
    period_d1=0 
    day1=pd.Timestamp(df['emergence'][i])
    Tspdate=(pd.Timestamp(df["transplanting"][i])).date().strftime('%Y-%m-%d')
    DOY=df['DOY1'][i]

    for kk in range(period1+1):   #开始该生长季内的逐日循环
        daystr1=(day1+pd.Timedelta(kk,unit='d')).date().strftime('%Y-%m-%d')
        DL=daylength(41.6667,DOY)
        metedf=mete[mete.index==daystr1]
        HU = float(metedf["DTT"])
        if daystr1 == Tspdate:
            CROPSTA = 3
        elif daystr1 > Tspdate:
            CROPSTA = 4

        if DVSRate >= 0. and DVSRate < 0.40:
            DVR = DVRJ * HU

        if DVSRate >= 0.40 and DVSRate < 0.65:
            PPFAC = 0.
            if DL < MOPP:
                PPFAC = 1.
            else :
                PPFAC = 1. - (DL - MOPP) * PPSE
            PPFAC = np.min([1.,np.max([0., PPFAC])])
            DVR   = DVRI * HU * PPFAC

        if DVSRate >= 0.65 and DVSRate < 1.00 :
            DVR = DVRP * HU
        
        if DVSRate >= 1.00 and DVSRate < 2.00 :
            DVR = DVRR * HU
        
        if(CROPSTA == 3):
            TSTR = TS
            TSHCKD = SHCKD*TSTR+TSTR

        if(CROPSTA > 3 and TS < (TSHCKD) ):
            DVR = 0.
        #print("日期：",daystr1,
#              "CROPSTA = ",CROPSTA,"TS = ", TS, "TSHCKD = ",TSHCKD
#               "DVR=",DVR,"DVSRate = ",DVSRate)
        TS = TS + HU 
        DVSRate = DVSRate + DVR
        period_d1+=1
        DOY+=1

        if (DVSRate >= 2.0 ):
            #print("抽穗日：",daystr1,period_d1,"到达抽穗日的DVS：",DVSRate)
            ErrorALL = period_d1-df["phase_ALL"][i]
            f1["误差"].append(ErrorALL)
            #print(Error)
            break
    if (DVSRate < 1.0 ):
        f1["误差"].append(np.nan)
        #生长季内的逐日循环结束
    f1["年份"].append(df["年份"][i])
F1=pd.DataFrame(f1)
F1.to_csv('D:/Wu/54362/ErrorALL.csv',index=False,encoding='gbk')
if F1.shape[0]!=0: 
    s=0
    ny=len(F1)
    for y in F1["误差"]:
        s+=y**2
    RMSE=(s/F1.shape[0])**0.5  
#     TSumJ = 1/DVRJ      #TSum in BVP phase
#     TSumI = 1/DVRI      #TSum in PVP phase
#     TSumP = 1/DVRP      #TSum in PFP phase
#     f2["TSum_BVP"].append(TSumJ_1)
#     f2["TSum_PVP"].append(TSumI_1)
#     f2["TSum_PFP"].append(TSumP_1)
    print(RMSE,ny)        


8.84916336766885 26
